<h2 style="color:blue" align="center">Plain Deep Neural Network (DNN)</h2>

#### Import the necessary libraries

In [51]:
import tensorflow as tf 
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import time, math 

from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras 
from keras.models import Sequential
from keras.layers import Dense 
from keras.callbacks import Callback, CSVLogger 

#### Load and split the dataset into training and testing set

In [52]:
(X_train,y_train),(X_test,y_test) = keras.datasets.boston_housing.load_data(path="boston_housing.npz", test_split=0.2, seed=3)

#### Normalize the input dataset

In [53]:
mms1 = MinMaxScaler()
mms1.fit(X_train)
X_train_normalized = mms1.transform(X_train)

mms2 = MinMaxScaler()
mms2.fit(X_test)
X_test_normalized = mms2.transform(X_test) 

#### Define DNN model with two layers, optimizer, metrics, and loss function

In [54]:
import random as python_random
python_random.seed(3)
np.random.seed(7)
tf.random.set_seed(13)

def get_model(): 
    model = Sequential([
        # input layer 784 neurons to first hidden layer with 64 neurons
        Dense(64, input_dim = 13, activation='relu'), 
        # first hidden layer to second hidden layer
        Dense(64, activation='relu'),  
        # Output layer with 10 neurons
        Dense(1, activation='linear')
    ])
    
    # General SGD
    #opti = keras.optimizers.SGD(learning_rate=0.01)
    
    # SGD with momentum
    #opti = keras.optimizers.SGD(learning_rate=0.01, momentum=0.6)
    
    # SGD with Nesterov momentum 
    #opti = keras.optimizers.SGD(learning_rate=0.01, momentum=0.6, nesterov=True)
    
    # RMSprop 
    #opti = keras.optimizers.RMSprop(learning_rate=0.001, momentum=0.6)
    
    # Adam
    opti = keras.optimizers.Adam(learning_rate=0.001) 
    
    # Adamax
    #opti = keras.optimizers.Adamax(learning_rate=0.001) 
    
    model.compile(
        optimizer = opti,
        loss = 'mean_squared_error',
        metrics = ['mae']
    )
    
    return model

#### Custom callbacks

For HM based optimizer model

In [55]:
class CustomCallbackHM(Callback):     
    initial_weights = 0
    previous_weights = 0
    call_hm = 0 
     
    def on_train_begin(self, logs=None):
        self.initial_weights = model_hm.get_weights() 
        self.initial_weights = np.array(self.initial_weights,dtype=object)
        self.previous_weights = self.initial_weights
        # Harmonic mean based weights calculation
        self.call_hm = np.vectorize(self.apply_hm)  
             
    def on_epoch_end(self, epoch, logs={}): 
        num_layers = len(model_hm.layers)  
        current_weights = model_hm.get_weights()
        current_weights = np.array(current_weights,dtype=object)        

        for i in range(num_layers):  
            # Harmonic mean based weights calculation
            tensor1 = tf.convert_to_tensor(self.previous_weights[i*2])
            tensor2 = tf.convert_to_tensor(current_weights[i*2])
            current_weights[i*2] = self.call_hm(tensor1, tensor2)   
            
        # Updating the model with new weights
        model_hm.set_weights(current_weights.tolist())
        self.previous_weights = current_weights
        
    def apply_hm(self, v1, v2):     
        if v1==0 or v2==0:
            return v2
        elif v1>0 and v2>0:
            hm = 2*v1*v2/(v1+v2)
            min1 = min(v1,v2)
            diff = abs(hm-min1)
            if v2 > v1:
                return v2 + diff
            else:
                return v2 - diff
        elif v1<0 and v2<0:
            hm = 2*v1*v2/(v1+v2)
            max1 = max(v1,v2)
            diff = abs(hm-max1)
            if v2 > v1:
                return v2 + diff
            else:
                return v2 - diff
        else:
            return v2   

To record loss and accuracy in CSV file

In [56]:
logger_generic_model = CSVLogger('Generic_model_MNIST.csv', append=False, separator=',')
logger_hm_model = CSVLogger('HM_model_MNIST.csv', append=False, separator=',')

#### Training

Get a model to assign same weights to model with and without HM

In [57]:
model = get_model() 
weights = model.get_weights() 
model_wihtout_hm = get_model()
model_wihtout_hm.set_weights(weights) 
model_hm = get_model()
model_hm.set_weights(weights) 

Generic opimizer model

In [58]:
st = time.time()
model_wihtout_hm.fit(X_train_normalized, y_train, epochs = 500, verbose=1, callbacks=[logger_generic_model], batch_size=404) 
et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

Epoch 1/500
1/1 [==============================] - 0s 302ms/step - loss: 602.7460 - mae: 22.7189
Epoch 2/500
1/1 [==============================] - 0s 7ms/step - loss: 600.1111 - mae: 22.6582
Epoch 3/500
1/1 [==============================] - 0s 9ms/step - loss: 597.4994 - mae: 22.5978
Epoch 4/500
1/1 [==============================] - 0s 7ms/step - loss: 594.9070 - mae: 22.5376
Epoch 5/500
1/1 [==============================] - 0s 7ms/step - loss: 592.3234 - mae: 22.4774
Epoch 6/500
1/1 [==============================] - 0s 7ms/step - loss: 589.7321 - mae: 22.4166
Epoch 7/500
1/1 [==============================] - 0s 7ms/step - loss: 587.1105 - mae: 22.3547
Epoch 8/500
1/1 [==============================] - 0s 7ms/step - loss: 584.4413 - mae: 22.2915
Epoch 9/500
1/1 [==============================] - 0s 7ms/step - loss: 581.7036 - mae: 22.2265
Epoch 10/500
1/1 [==============================] - 0s 7ms/step - loss: 578.8874 - mae: 22.1593
Epoch 11/500
1/1 [=============================

1/1 [==============================] - 0s 7ms/step - loss: 64.7868 - mae: 5.8050
Epoch 173/500
1/1 [==============================] - 0s 8ms/step - loss: 64.4540 - mae: 5.7873
Epoch 174/500
1/1 [==============================] - 0s 8ms/step - loss: 64.1272 - mae: 5.7700
Epoch 175/500
1/1 [==============================] - 0s 7ms/step - loss: 63.8055 - mae: 5.7535
Epoch 176/500
1/1 [==============================] - 0s 7ms/step - loss: 63.4901 - mae: 5.7371
Epoch 177/500
1/1 [==============================] - 0s 7ms/step - loss: 63.1803 - mae: 5.7213
Epoch 178/500
1/1 [==============================] - 0s 7ms/step - loss: 62.8769 - mae: 5.7057
Epoch 179/500
1/1 [==============================] - 0s 8ms/step - loss: 62.5796 - mae: 5.6904
Epoch 180/500
1/1 [==============================] - 0s 7ms/step - loss: 62.2887 - mae: 5.6751
Epoch 181/500
1/1 [==============================] - 0s 8ms/step - loss: 62.0032 - mae: 5.6599
Epoch 182/500
1/1 [==============================] - 0s 7ms/step

1/1 [==============================] - 0s 8ms/step - loss: 30.1477 - mae: 3.8157
Epoch 345/500
1/1 [==============================] - 0s 7ms/step - loss: 30.0036 - mae: 3.8053
Epoch 346/500
1/1 [==============================] - 0s 8ms/step - loss: 29.8605 - mae: 3.7951
Epoch 347/500
1/1 [==============================] - 0s 7ms/step - loss: 29.7184 - mae: 3.7849
Epoch 348/500
1/1 [==============================] - 0s 8ms/step - loss: 29.5772 - mae: 3.7745
Epoch 349/500
1/1 [==============================] - 0s 7ms/step - loss: 29.4370 - mae: 3.7640
Epoch 350/500
1/1 [==============================] - 0s 7ms/step - loss: 29.2979 - mae: 3.7534
Epoch 351/500
1/1 [==============================] - 0s 8ms/step - loss: 29.1597 - mae: 3.7427
Epoch 352/500
1/1 [==============================] - 0s 7ms/step - loss: 29.0226 - mae: 3.7320
Epoch 353/500
1/1 [==============================] - 0s 7ms/step - loss: 28.8866 - mae: 3.7215
Epoch 354/500
1/1 [==============================] - 0s 7ms/step

HM based optimizer model

In [59]:
st = time.time()
model_hm.fit(X_train_normalized, y_train, epochs = 357, verbose=1, callbacks=[CustomCallbackHM(),logger_hm_model], batch_size=404) 
et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

Epoch 1/357
1/1 [==============================] - 0s 310ms/step - loss: 602.7460 - mae: 22.7189
Epoch 2/357
1/1 [==============================] - 0s 16ms/step - loss: 599.0299 - mae: 22.6330
Epoch 3/357
1/1 [==============================] - 0s 16ms/step - loss: 595.3619 - mae: 22.5478
Epoch 4/357
1/1 [==============================] - 0s 16ms/step - loss: 591.7150 - mae: 22.4625
Epoch 5/357
1/1 [==============================] - 0s 17ms/step - loss: 588.0449 - mae: 22.3759
Epoch 6/357
1/1 [==============================] - 0s 15ms/step - loss: 584.2986 - mae: 22.2870
Epoch 7/357
1/1 [==============================] - 0s 16ms/step - loss: 580.4243 - mae: 22.1945
Epoch 8/357
1/1 [==============================] - 0s 14ms/step - loss: 576.3970 - mae: 22.0982
Epoch 9/357
1/1 [==============================] - 0s 17ms/step - loss: 572.1907 - mae: 21.9970
Epoch 10/357
1/1 [==============================] - 0s 16ms/step - loss: 567.7795 - mae: 21.8902
Epoch 11/357
1/1 [====================

1/1 [==============================] - 0s 16ms/step - loss: 50.7057 - mae: 5.0486
Epoch 171/357
1/1 [==============================] - 0s 14ms/step - loss: 50.4602 - mae: 5.0359
Epoch 172/357
1/1 [==============================] - 0s 13ms/step - loss: 50.2150 - mae: 5.0237
Epoch 173/357
1/1 [==============================] - 0s 13ms/step - loss: 49.9698 - mae: 5.0116
Epoch 174/357
1/1 [==============================] - 0s 14ms/step - loss: 49.7247 - mae: 4.9994
Epoch 175/357
1/1 [==============================] - 0s 14ms/step - loss: 49.4797 - mae: 4.9871
Epoch 176/357
1/1 [==============================] - 0s 15ms/step - loss: 49.2347 - mae: 4.9747
Epoch 177/357
1/1 [==============================] - 0s 14ms/step - loss: 48.9900 - mae: 4.9622
Epoch 178/357
1/1 [==============================] - 0s 15ms/step - loss: 48.7454 - mae: 4.9498
Epoch 179/357
1/1 [==============================] - 0s 16ms/step - loss: 48.5009 - mae: 4.9374
Epoch 180/357
1/1 [==============================] - 0

1/1 [==============================] - 0s 14ms/step - loss: 19.5857 - mae: 3.0386
Epoch 341/357
1/1 [==============================] - 0s 17ms/step - loss: 19.5245 - mae: 3.0338
Epoch 342/357
1/1 [==============================] - 0s 14ms/step - loss: 19.4642 - mae: 3.0287
Epoch 343/357
1/1 [==============================] - 0s 13ms/step - loss: 19.4046 - mae: 3.0231
Epoch 344/357
1/1 [==============================] - 0s 13ms/step - loss: 19.3457 - mae: 3.0173
Epoch 345/357
1/1 [==============================] - 0s 13ms/step - loss: 19.2882 - mae: 3.0114
Epoch 346/357
1/1 [==============================] - 0s 14ms/step - loss: 19.2316 - mae: 3.0059
Epoch 347/357
1/1 [==============================] - 0s 14ms/step - loss: 19.1754 - mae: 3.0009
Epoch 348/357
1/1 [==============================] - 0s 15ms/step - loss: 19.1195 - mae: 2.9961
Epoch 349/357
1/1 [==============================] - 0s 14ms/step - loss: 19.0644 - mae: 2.9914
Epoch 350/357
1/1 [==============================] - 0

Model summary

In [60]:
model_hm.summary()

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_45 (Dense)            (None, 64)                896       
                                                                 
 dense_46 (Dense)            (None, 64)                4160      
                                                                 
 dense_47 (Dense)            (None, 1)                 65        
                                                                 
Total params: 5,121
Trainable params: 5,121
Non-trainable params: 0
_________________________________________________________________


###### Testing the model

Generic opimizer model

In [61]:
model_wihtout_hm.evaluate(X_test_normalized, y_test)

4/4 [==============================] - 0s 2ms/step - loss: 20.2916 - mae: 3.0116


[20.291616439819336, 3.011594295501709]

In [62]:
input_data = np.array([0.63796, 0.00, 8.140, 0, 0.5380, 6.0960, 84.50, 4.4619, 4, 307.0, 21.00, 380.02, 10.26])
input_data_transformed = mms1.transform(input_data.reshape(1, -1))
output = model_wihtout_hm.predict(input_data_transformed) 
output[0][0]

1/1 [==============================] - 0s 46ms/step


19.983803

HM based optimizer model

In [63]:
model_hm.evaluate(X_test_normalized, y_test)

4/4 [==============================] - 0s 1ms/step - loss: 20.5012 - mae: 2.9848


[20.501174926757812, 2.9848132133483887]

In [64]:
output = model_hm.predict(input_data_transformed) 
output[0][0]

1/1 [==============================] - 0s 43ms/step


19.890657